## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [9]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [10]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [11]:
# count
rdd.count()

1000

In [12]:
# first
rdd.first()

1

In [13]:
# take
rdd.take(8)

[1, 2, 3, 4, 5, 6, 7, 8]

In [14]:
# top
rdd.top(8)

[1000, 999, 998, 997, 996, 995, 994, 993]

In [15]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [16]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([8.87505270e-01, 6.34402898e-01, 2.50838960e+00, 2.40996844e+00,
       3.23420009e-01, 2.91383072e+00, 8.55445900e-02, 2.54212387e+00,
       4.98384169e+00, 7.50604483e+00, 1.10760345e+00, 1.48061628e+00,
       1.25207272e+01, 1.02197500e+01, 1.11081198e+01, 1.54128925e+01,
       9.18273440e+00, 1.46463514e+01, 6.97334032e-01, 1.58434237e+01,
       1.93744420e+01, 1.78806260e+01, 2.05055017e+01, 1.78592400e+01,
       1.89986813e+01, 2.46115032e+01, 2.37280330e+01, 1.52256530e+01,
       2.64455719e+01, 8.17650197e+00, 3.29242542e+00, 2.66521311e+01,
       2.51883709e+01, 2.05618167e+01, 1.32230356e+01, 1.70689961e+01,
       9.99918031e+00, 3.73109886e-01, 1.81921669e+01, 3.87586660e+01,
       4.05248192e+01, 3.61510552e+01, 2.12664984e+01, 3.70902089e+01,
       1.49568314e+01, 3.40442819e+01, 3.20581711e+01, 7.28514600e+00,
       4.49283466e+00, 3.17099863e+01, 1.49101988e+01, 4.72119963e+01,
       1.34577095e+01, 1.11421405e+01, 5.46623423e+01, 3.86202517e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [17]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(10)

[954.8620164279081,
 922.4445383496046,
 908.8912127390508,
 892.4964710783845,
 848.5264150192202,
 845.0122104641296,
 839.9660741763034,
 827.2176338031853,
 814.0634836563104,
 810.3492630244049]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [18]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [20]:
# perform action to retrieve rdd values
revenue_minus_tax.top(10)

[878.4730551136755,
 848.6489752816362,
 836.1799157199268,
 821.0967533921138,
 780.6443018176826,
 777.4112336269992,
 772.7687882421992,
 761.0402230989305,
 748.9384049638056,
 745.5213219824525]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [21]:
discounted = revenue_minus_tax.map(lambda x: x*0.9)

In [22]:
discounted.take(10)

[0.7348543638174783,
 0.525285599385315,
 2.0769465902082063,
 1.9954538702450408,
 0.2677917670389804,
 2.4126518320969224,
 0.0708309204847682,
 2.104878562610808,
 4.126620918929821,
 6.215005116994288]

In [28]:
discounted.count()

1000

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [23]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[790.625749602308,
 763.7840777534726,
 752.5619241479342,
 738.9870780529025,
 702.5798716359144,
 699.6701102642993,
 695.4919094179793,
 684.9362007890375,
 674.0445644674251,
 670.9691897842073,
 668.6981457767771,
 660.3856380735344,
 659.6211651856045,
 653.8987171669164,
 645.6081566533207]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [24]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [25]:
mapped = price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.887505270310964, 0.7348543638174783), (0.6344028978083514, 0.525285599385315), (2.508389601700732, 2.0769465902082063), (2.4099684423249284, 1.9954538702450408), (0.323420008501184, 0.2677917670389804), (2.9138307150929013, 2.4126518320969224), (0.08554458995744951, 0.0708309204847682), (2.542123867887449, 2.104878562610808), (4.983841689528769, 4.126620918929821), (7.506044827287787, 6.215005116994288)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [26]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.887505270310964, 0.7348543638174783, 0.6344028978083514, 0.525285599385315, 2.508389601700732, 2.0769465902082063, 2.4099684423249284, 1.9954538702450408, 0.323420008501184, 0.2677917670389804]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [27]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

264

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [29]:
selected_items.reduce(lambda x,y: x+y)

120144.96165555369

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [30]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(8, 312.61193567333487),
 (35, 309.98030983088756),
 (48, 306.8477105415575),
 (48, 313.73366831761234),
 (2, 300.6761740560051),
 (27, 341.10641132926025),
 (41, 331.268114237132)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [31]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x+y)

In [32]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(28, 4508.550208249191),
 (36, 4431.476734247239),
 (12, 4089.834600887949),
 (6, 3822.603706712932),
 (47, 3786.186491907878),
 (17, 3584.055699542529),
 (15, 3546.372252358511),
 (14, 3522.2363510670443),
 (20, 3435.5833887749304),
 (11, 3408.167711224666),
 (49, 3354.644695773114),
 (45, 3235.0777004016018),
 (27, 3103.1267477488177),
 (4, 3047.754350911592),
 (30, 2855.0931884719594),
 (31, 2785.3728037125607),
 (44, 2717.2142144601107),
 (24, 2651.8313423219465),
 (29, 2584.665459403255),
 (33, 2552.049739089919),
 (50, 2485.585897611878),
 (41, 2444.3241081335054),
 (2, 2347.107920366195),
 (26, 2289.455803306033),
 (37, 2242.259416392463),
 (25, 2222.858840593794),
 (18, 2214.312511283935),
 (13, 2132.395938961333),
 (34, 2120.6667453695563),
 (1, 2095.3974628195515),
 (35, 2081.2404268228693),
 (16, 1862.696195395639),
 (48, 1842.4922085173564),
 (8, 1835.1848055997407),
 (3, 1812.2991110511107),
 (38, 1803.2497209071885),
 (23, 1730.2136068779564),
 (32, 1711.4848080160652),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [33]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda kv: kv[1], reverse=True)

[(18, 12),
 (42, 12),
 (48, 9),
 (1, 9),
 (17, 9),
 (3, 8),
 (39, 8),
 (8, 8),
 (19, 8),
 (50, 7),
 (35, 7),
 (10, 7),
 (7, 7),
 (28, 6),
 (12, 6),
 (36, 6),
 (38, 6),
 (33, 6),
 (16, 6),
 (22, 5),
 (23, 5),
 (2, 5),
 (15, 5),
 (27, 5),
 (41, 5),
 (40, 5),
 (32, 5),
 (6, 5),
 (20, 5),
 (4, 5),
 (30, 4),
 (5, 4),
 (14, 4),
 (47, 4),
 (31, 4),
 (37, 4),
 (9, 4),
 (21, 3),
 (24, 3),
 (29, 3),
 (45, 3),
 (13, 3),
 (43, 3),
 (34, 3),
 (11, 3),
 (26, 3),
 (44, 2),
 (49, 2),
 (25, 2),
 (46, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
